In [82]:
%load_ext line_profiler
import pandas as pd
import numpy as np
import networkx as nx

from bb_binary import Repository,load_frame_container
from pandas import DataFrame, Series
from collections import namedtuple
from scipy import spatial

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [83]:
def get_files(path):
    repo = Repository(path)
    file = list(repo.iter_fnames())
    a = [f.split('/')[-1].split("_")[1] for f in file]
    l = len(a)/4
    npa = np.array(file).reshape(l,4)
    return npa

In [84]:
def get_dataframe(fc):
    detection = namedtuple('Detection', ['idx','xpos','ypos',
        'radius','decodedId', 'frame_idx', 'timestamp', 'cam_id', 'fc_id'])

    l = []
    for f in fc.frames:
        tpls = [detection(d.idx, d.xpos, d.ypos, d.radius, list(d.decodedId),
            f.frameIdx, f.timestamp, fc.camId, fc.id)
            for d in f.detectionsUnion.detectionsDP]
        l.append(pd.DataFrame(tpls))
    return pd.concat(l)

In [85]:
def calcIds(df, threshold):
    # print('\n### Calc IDs with threshold: {}'.format(threshold))
    #print('#Detections before calcualting IDs: {}'.format(df.shape[0]))

    # calc confidence value
        # 0...256 in 0...1 umrechnen
        # fuer jedes bit abstand zu 0.5 berechnen und dann minimum behalten
    # add confidence value to dataframe as column
    df = df.assign(confidence = df.decodedId.apply(get_confidence))

    # die detections entfernen die nicht  die nicht gut genug sind
    df = df[df.confidence >= threshold]

    # fuer den Rest der ueber bleibt die ID berechnen und an DF anhaengen
    df = df.assign(id = df.decodedId.apply(get_detected_id))

    df = df.drop('decodedId', 1)

    #print('Number of Detections after calcualting IDs: {}'.format(df.shape[0]))
    return df

In [86]:
def get_detected_id(bits):
    # Umrechnen in binary array [0,1,1,1,0,1,1,1,0,0,0,1]
    bits = np.array(bits)
    bits = bits/255
    binary_id = [int(x > 0.5) for x in bits]

    decimal_id = int(''.join([str(c) for c in binary_id[:11]]), 2)

    # determine what kind of parity bit was used and add 2^11 to decimal id
    # uneven parity bit was used
    if ((sum(binary_id) % 2) == 1):
        decimal_id += 2048

    return decimal_id

In [87]:
def get_confidence(bits):
    # 12 bits mit Werten zwischen 0 und 256
    bits = np.array(bits)
    bits = bits/255

    return np.min(np.abs(0.5 - bits)) * 2

In [88]:
def get_close_bees(df, distance):

    df = df.reset_index(level = 'frame_idx')

    m = pd.merge(df, df, on='frame_idx')
    #m = m.query('id_x < id_y')
    m = m[m.id_x < m.id_y]

    m.loc[:, 'dist'] = np.square(m.xpos_x - m.xpos_y) \
        + np.square(m.ypos_x - m.ypos_y)

    filtered = m[m.dist <= distance**2]

    filtered = filtered[['frame_idx','id_x', 'id_y']]
    return filtered

In [89]:
def get_close_bees_kd(df, distance):

    df_close = DataFrame()

    gr = df.groupby('frame_idx')

    for i, group in gr:
        xy_coordinates = group[['xpos', 'ypos']].values
        tree = spatial.KDTree(xy_coordinates, leafsize=20)
        result = tree.query_pairs(distance)
        l = [[i,group['id'].iat[a], group['id'].iat[b]] for a,b in result]
        df_close = df_close.append(DataFrame(l, columns=['frame_idx', 'id_x', 'id_y']))

In [90]:
def bee_pairs_to_timeseries(df):
    close = df[['frame_idx', 'id_x', 'id_y']]
    close = close.set_index(['frame_idx'])
    close['pair'] = list(zip(close.id_x, close.id_y))
    u_pairs = close.pair.unique()
    dft = DataFrame(0, index=u_pairs, columns=np.arange(1024))
    gr = close.groupby(level='frame_idx')

    for i, group in gr:
        l = group['pair']
        dft.loc[l,i] = 1

    return dft

In [91]:
def get_ketten(kette, val):
    kette = kette.apply(str)
    s = kette.str.cat(sep='')
    ss = s.split(val)
    return [x for x in ss if len(x) > 0]

In [92]:
def extract_interactions(dft, minlength):
    kette = dft.apply(get_ketten, axis=1, args=["0"])
    kk = kette.apply(lambda x:[len(item) for item in x])
    kk = kk.apply(lambda x: len([item for item in x if item >= minlength]))
    return kk[kk > 0]

In [100]:
# 0.9, 160, "08", 3
def f(c,d,m,l):
    filename = "{}month-{}dist-{}conf-{}len-1h".format(m,d,str(c).replace('.',''),l)

    f = "../00_Data/testset_2015_1h/"
    p = "2015" + m + "2215"
    CONFIDENCE = c
    DISTANCE = d
    xmax = 3000
    ymax = 4000
    LENGTH = l
    path = f+p

    files = get_files(path)

    interactions = Series()

    for file_list in files[:1]:
        
        dataframes = np.empty(4, dtype=object)
        
        for i in list(range(4)):
            fc = load_frame_container(file_list[i])
            df = get_dataframe(fc)
            df = calcIds(df,CONFIDENCE)

            camIdx = file_list[i].split("/")[-1].split("_")[1]
            dataframes[camIdx] = df
        
        # cam 0 und cam1 nach rechts verschieben
        dataframes[0].xpos = dataframes[0].xpos + xmax
        dataframes[1].xpos = dataframes[1].xpos + xmax

        # Seiten zusammenfugen
        side0 = pd.concat([dataframes[3], dataframes[0]])
        side1 = pd.concat([dataframes[2], dataframes[1]])

        close1 = get_close_bees(side0, DISTANCE)
        close2 = get_close_bees(side1, DISTANCE)
        
        close1kd = get_close_bees_kd(side0, DISTANCE)
        close2kd = get_close_bees_kd(side1, DISTANCE)

        close = pd.concat([close1,close2])
        
        p = bee_pairs_to_timeseries(close)

        edges = extract_interactions(p,LENGTH)
        
        interactions = pd.concat([interactions, edges])

    G = create_graph2(interactions)
    print(nx.info(G))

    #nx.write_graphml(G, filename + ".graphml")

In [97]:
def create_graph2(pairs):
    G = nx.Graph()

    for elem in pairs.iteritems():
        G.add_edge(int(elem[0][0]), int(elem[0][1]), weight=int(elem[1]))

    return G

In [113]:
%lprun -f f -f get_dataframe -f get_close_bees_kd -f get_close_bees f(0.9, 160, "09", 3)

/usr/local/lib/python3.5/site-packages/ipykernel/__main__.py:6: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python3.5/site-packages/ipykernel/__main__.py:30: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


Name: 
Type: Graph
Number of nodes: 240
Number of edges: 1937
Average degree:  16.1417


In [104]:
side0.head()

,idx,xpos,ypos,radius,frame_idx,timestamp,cam_id,fc_id,confidence,id
3,3,452,2450,22.891178,0,1.440256e+09,3,17639613505007922451,0.921569,3994
4,4,702,3464,22.759066,0,1.440256e+09,3,17639613505007922451,0.968627,2251
5,5,887,2225,22.952803,0,1.440256e+09,3,17639613505007922451,0.992157,564
7,7,962,1886,23.071014,0,1.440256e+09,3,17639613505007922451,0.984314,3700
9,9,1237,3374,22.624674,0,1.440256e+09,3,17639613505007922451,1.000000,2762


In [114]:
side0.groupby('frame_idx').size().mean()

77.5146484375

In [115]:
side1.groupby('frame_idx').size().mean()


56.1884765625

In [109]:
287*8.7/100

24.968999999999998

In [ ]:
# Oktober

Timer unit: 1e-06 s

Total time: 32.0535 s
File: <ipython-input-100-c9073bcbd3cc>
Function: f at line 2

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     2                                           def f(c,d,m,l):
     3                                               global side0
     4                                               global side1
     5         1           12     12.0      0.0      filename = "{}month-{}dist-{}conf-{}len-1h".format(m,d,str(c).replace('.',''),l)
     6                                           
     7         1            2      2.0      0.0      f = "../00_Data/testset_2015_1h/"
     8         1            3      3.0      0.0      p = "2015" + m + "2215"
     9         1            2      2.0      0.0      CONFIDENCE = c
    10         1            2      2.0      0.0      DISTANCE = d
    11         1            2      2.0      0.0      xmax = 3000
    12         1            1      1.0      0.0      ymax = 4000
    13         1            1      1.0      0.0      LENGTH = l
    14         1            1      1.0      0.0      path = f+p
    15                                           
    16         1        23027  23027.0      0.1      files = get_files(path)
    17                                           
    18         1          417    417.0      0.0      interactions = Series()
    19                                           
    20         2           10      5.0      0.0      for file_list in files[:1]:
    21                                                   
    22         1            8      8.0      0.0          dataframes = np.empty(4, dtype=object)
    23                                                   
    24         5           16      3.2      0.0          for i in list(range(4)):
    25         4        35299   8824.8      0.1              fc = load_frame_container(file_list[i])
    26         4     18335109 4583777.2     57.2              df = get_dataframe(fc)
    27         4      2250694 562673.5      7.0              df = calcIds(df,CONFIDENCE)
    28                                           
    29         4          129     32.2      0.0              camIdx = file_list[i].split("/")[-1].split("_")[1]
    30         4          279     69.8      0.0              dataframes[camIdx] = df
    31                                                   
    32                                                   # cam 0 und cam1 nach rechts verschieben
    33         1         1369   1369.0      0.0          dataframes[0].xpos = dataframes[0].xpos + xmax
    34         1         1209   1209.0      0.0          dataframes[1].xpos = dataframes[1].xpos + xmax
    35                                           
    36                                                   # Seiten zusammenfugen
    37         1        11624  11624.0      0.0          side0 = pd.concat([dataframes[3], dataframes[0]])
    38         1        13377  13377.0      0.0          side1 = pd.concat([dataframes[2], dataframes[1]])
    39                                           
    40         1       208182 208182.0      0.6          close1 = get_close_bees(side0, DISTANCE)
    41         1       201201 201201.0      0.6          close2 = get_close_bees(side1, DISTANCE)
    42                                                   
    43         1      4870109 4870109.0     15.2          close1kd = get_close_bees_kd(side0, DISTANCE)
    44         1      4890696 4890696.0     15.3          close2kd = get_close_bees_kd(side1, DISTANCE)
    45                                           
    46         1          826    826.0      0.0          close = pd.concat([close1,close2])
    47                                                   
    48         1      1087414 1087414.0      3.4          p = bee_pairs_to_timeseries(close)
    49                                           
    50         1       121118 121118.0      0.4          edges = extract_interactions(p,LENGTH)
    51                                                   
    52         1          623    623.0      0.0          interactions = pd.concat([interactions, edges])
    53                                           
    54         1          471    471.0      0.0      G = create_graph2(interactions)
    55         1          267    267.0      0.0      print(nx.info(G))

Total time: 18.2431 s
File: <ipython-input-84-29acb5835905>
Function: get_dataframe at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def get_dataframe(fc):
     2         4            5      1.2      0.0      detection = namedtuple('Detection', ['idx','xpos','ypos',
     3         4         4218   1054.5      0.0          'radius','decodedId', 'frame_idx', 'timestamp', 'cam_id', 'fc_id'])
     4                                           
     5         4            6      1.5      0.0      l = []
     6      4100        17180      4.2      0.1      for f in fc.frames:
     7      4096         6312      1.5      0.0          tpls = [detection(d.idx, d.xpos, d.ypos, d.radius, list(d.decodedId),
     8                                                       f.frameIdx, f.timestamp, fc.camId, fc.id)
     9      4096       753914    184.1      4.1              for d in f.detectionsUnion.detectionsDP]
    10      4096     10282642   2510.4     56.4          l.append(pd.DataFrame(tpls))
    11         4      7178851 1794712.8     39.4      return pd.concat(l)

Total time: 0.408349 s
File: <ipython-input-88-f3f21f8368b9>
Function: get_close_bees at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def get_close_bees(df, distance):
     2                                           
     3         2         5386   2693.0      1.3      df = df.reset_index(level = 'frame_idx')
     4                                           
     5         2        91075  45537.5     22.3      m = pd.merge(df, df, on='frame_idx')
     6                                               #m = m.query('id_x < id_y')
     7         2       166137  83068.5     40.7      m = m[m.id_x < m.id_y]
     8                                           
     9         2       107548  53774.0     26.3      m.loc[:, 'dist'] = np.square(m.xpos_x - m.xpos_y)         + np.square(m.ypos_x - m.ypos_y)
    10                                           
    11         2        36247  18123.5      8.9      filtered = m[m.dist <= distance**2]
    12                                           
    13         2         1954    977.0      0.5      filtered = filtered[['frame_idx','id_x', 'id_y']]
    14         2            2      1.0      0.0      return filtered

Total time: 9.74399 s
File: <ipython-input-89-d3df72123a10>
Function: get_close_bees_kd at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def get_close_bees_kd(df, distance):
     2                                           
     3         2         1213    606.5      0.0      df_close = DataFrame()
     4                                           
     5         2          696    348.0      0.0      gr = df.groupby('frame_idx')
     6                                           
     7      2050       693392    338.2      7.1      for i, group in gr:
     8      2048      1972924    963.3     20.2          xy_coordinates = group[['xpos', 'ypos']].values
     9      2048       107955     52.7      1.1          tree = spatial.KDTree(xy_coordinates, leafsize=20)
    10      2048      1751640    855.3     18.0          result = tree.query_pairs(distance)
    11      2048       224536    109.6      2.3          l = [[i,group['id'].iat[a], group['id'].iat[b]] for a,b in result]
    12      2048      4991637   2437.3     51.2          df_close = df_close.append(DataFrame(l, columns=['frame_idx', 'id_x', 'id_y']))

In [ ]:
#side0 = 14.7685546875
#side1 = 15.0341796875

In [ ]:
# August

Timer unit: 1e-06 s

Total time: 287.559 s
File: <ipython-input-100-c9073bcbd3cc>
Function: f at line 2

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     2                                           def f(c,d,m,l):
     3                                               global side0
     4                                               global side1
     5         1           18     18.0      0.0      filename = "{}month-{}dist-{}conf-{}len-1h".format(m,d,str(c).replace('.',''),l)
     6                                           
     7         1            2      2.0      0.0      f = "../00_Data/testset_2015_1h/"
     8         1            4      4.0      0.0      p = "2015" + m + "2215"
     9         1            2      2.0      0.0      CONFIDENCE = c
    10         1            2      2.0      0.0      DISTANCE = d
    11         1            2      2.0      0.0      xmax = 3000
    12         1            2      2.0      0.0      ymax = 4000
    13         1            3      3.0      0.0      LENGTH = l
    14         1            2      2.0      0.0      path = f+p
    15                                           
    16         1        22838  22838.0      0.0      files = get_files(path)
    17                                           
    18         1          720    720.0      0.0      interactions = Series()
    19                                           
    20         2           23     11.5      0.0      for file_list in files[:1]:
    21                                                   
    22         1           15     15.0      0.0          dataframes = np.empty(4, dtype=object)
    23                                                   
    24         5           20      4.0      0.0          for i in list(range(4)):
    25         4        97354  24338.5      0.0              fc = load_frame_container(file_list[i])
    26         4     24928450 6232112.5      8.7              df = get_dataframe(fc)
    27         4     23481809 5870452.2      8.2              df = calcIds(df,CONFIDENCE)
    28                                           
    29         4          154     38.5      0.0              camIdx = file_list[i].split("/")[-1].split("_")[1]
    30         4          312     78.0      0.0              dataframes[camIdx] = df
    31                                                   
    32                                                   # cam 0 und cam1 nach rechts verschieben
    33         1         1628   1628.0      0.0          dataframes[0].xpos = dataframes[0].xpos + xmax
    34         1         1090   1090.0      0.0          dataframes[1].xpos = dataframes[1].xpos + xmax
    35                                           
    36                                                   # Seiten zusammenfugen
    37         1        18214  18214.0      0.0          side0 = pd.concat([dataframes[3], dataframes[0]])
    38         1        11540  11540.0      0.0          side1 = pd.concat([dataframes[2], dataframes[1]])
    39                                           
    40         1     93542932 93542932.0     32.5          close1 = get_close_bees(side0, DISTANCE)
    41         1     13805993 13805993.0      4.8          close2 = get_close_bees(side1, DISTANCE)
    42                                                   
    43         1     69713737 69713737.0     24.2          close1kd = get_close_bees_kd(side0, DISTANCE)
    44         1     30214080 30214080.0     10.5          close2kd = get_close_bees_kd(side1, DISTANCE)
    45                                           
    46         1        40285  40285.0      0.0          close = pd.concat([close1,close2])
    47                                                   
    48         1     13913490 13913490.0      4.8          p = bee_pairs_to_timeseries(close)
    49                                           
    50         1     17709734 17709734.0      6.2          edges = extract_interactions(p,LENGTH)
    51                                                   
    52         1         1208   1208.0      0.0          interactions = pd.concat([interactions, edges])
    53                                           
    54         1        50195  50195.0      0.0      G = create_graph2(interactions)
    55         1         2690   2690.0      0.0      print(nx.info(G))

Total time: 24.8314 s
File: <ipython-input-84-29acb5835905>
Function: get_dataframe at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def get_dataframe(fc):
     2         4            6      1.5      0.0      detection = namedtuple('Detection', ['idx','xpos','ypos',
     3         4         4715   1178.8      0.0          'radius','decodedId', 'frame_idx', 'timestamp', 'cam_id', 'fc_id'])
     4                                           
     5         4            7      1.8      0.0      l = []
     6      4100        15890      3.9      0.1      for f in fc.frames:
     7      4096         6123      1.5      0.0          tpls = [detection(d.idx, d.xpos, d.ypos, d.radius, list(d.decodedId),
     8                                                       f.frameIdx, f.timestamp, fc.camId, fc.id)
     9      4096      6788096   1657.2     27.3              for d in f.detectionsUnion.detectionsDP]
    10      4096     10566357   2579.7     42.6          l.append(pd.DataFrame(tpls))
    11         4      7450225 1862556.2     30.0      return pd.concat(l)

In [ ]:
#side0 216.3544921875
#side1 102.5791015625

Timer unit: 1e-06 s

Total time: 79.9249 s
File: <ipython-input-100-c9073bcbd3cc>
Function: f at line 2

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     2                                           def f(c,d,m,l):
     3                                               global side0
     4                                               global side1
     5         1           13     13.0      0.0      filename = "{}month-{}dist-{}conf-{}len-1h".format(m,d,str(c).replace('.',''),l)
     6                                           
     7         1            2      2.0      0.0      f = "../00_Data/testset_2015_1h/"
     8         1            2      2.0      0.0      p = "2015" + m + "2215"
     9         1            1      1.0      0.0      CONFIDENCE = c
    10         1            2      2.0      0.0      DISTANCE = d
    11         1            2      2.0      0.0      xmax = 3000
    12         1            1      1.0      0.0      ymax = 4000
    13         1            1      1.0      0.0      LENGTH = l
    14         1            1      1.0      0.0      path = f+p
    15                                           
    16         1        18338  18338.0      0.0      files = get_files(path)
    17                                           
    18         1          420    420.0      0.0      interactions = Series()
    19                                           
    20         2           16      8.0      0.0      for file_list in files[:1]:
    21                                                   
    22         1            7      7.0      0.0          dataframes = np.empty(4, dtype=object)
    23                                                   
    24         5           16      3.2      0.0          for i in list(range(4)):
    25         4        46918  11729.5      0.1              fc = load_frame_container(file_list[i])
    26         4     20345255 5086313.8     25.5              df = get_dataframe(fc)
    27         4      9663543 2415885.8     12.1              df = calcIds(df,CONFIDENCE)
    28                                           
    29         4          118     29.5      0.0              camIdx = file_list[i].split("/")[-1].split("_")[1]
    30         4          300     75.0      0.0              dataframes[camIdx] = df
    31                                                   
    32                                                   # cam 0 und cam1 nach rechts verschieben
    33         1         1159   1159.0      0.0          dataframes[0].xpos = dataframes[0].xpos + xmax
    34         1          837    837.0      0.0          dataframes[1].xpos = dataframes[1].xpos + xmax
    35                                           
    36                                                   # Seiten zusammenfugen
    37         1         9729   9729.0      0.0          side0 = pd.concat([dataframes[3], dataframes[0]])
    38         1        11839  11839.0      0.0          side1 = pd.concat([dataframes[2], dataframes[1]])
    39                                           
    40         1      4391051 4391051.0      5.5          close1 = get_close_bees(side0, DISTANCE)
    41         1      2073035 2073035.0      2.6          close2 = get_close_bees(side1, DISTANCE)
    42                                                   
    43         1     20611443 20611443.0     25.8          close1kd = get_close_bees_kd(side0, DISTANCE)
    44         1     14955564 14955564.0     18.7          close2kd = get_close_bees_kd(side1, DISTANCE)
    45                                           
    46         1         1605   1605.0      0.0          close = pd.concat([close1,close2])
    47                                                   
    48         1      3503268 3503268.0      4.4          p = bee_pairs_to_timeseries(close)
    49                                           
    50         1      4276282 4276282.0      5.4          edges = extract_interactions(p,LENGTH)
    51                                                   
    52         1          945    945.0      0.0          interactions = pd.concat([interactions, edges])
    53                                           
    54         1        11975  11975.0      0.0      G = create_graph2(interactions)
    55         1         1237   1237.0      0.0      print(nx.info(G))

Total time: 20.2554 s
File: <ipython-input-84-29acb5835905>
Function: get_dataframe at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def get_dataframe(fc):
     2         4            5      1.2      0.0      detection = namedtuple('Detection', ['idx','xpos','ypos',
     3         4         4128   1032.0      0.0          'radius','decodedId', 'frame_idx', 'timestamp', 'cam_id', 'fc_id'])
     4                                           
     5         4            8      2.0      0.0      l = []
     6      4100        15885      3.9      0.1      for f in fc.frames:
     7      4096         5935      1.4      0.0          tpls = [detection(d.idx, d.xpos, d.ypos, d.radius, list(d.decodedId),
     8                                                       f.frameIdx, f.timestamp, fc.camId, fc.id)
     9      4096      2854657    696.9     14.1              for d in f.detectionsUnion.detectionsDP]
    10      4096     10214124   2493.7     50.4          l.append(pd.DataFrame(tpls))
    11         4      7160687 1790171.8     35.4      return pd.concat(l)

Total time: 6.44989 s
File: <ipython-input-88-f3f21f8368b9>
Function: get_close_bees at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def get_close_bees(df, distance):
     2                                           
     3         2        31578  15789.0      0.5      df = df.reset_index(level = 'frame_idx')
     4                                           
     5         2      1461840 730920.0     22.7      m = pd.merge(df, df, on='frame_idx')
     6                                               #m = m.query('id_x < id_y')
     7         2      3504530 1752265.0     54.3      m = m[m.id_x < m.id_y]
     8                                           
     9         2       760400 380200.0     11.8      m.loc[:, 'dist'] = np.square(m.xpos_x - m.xpos_y)         + np.square(m.ypos_x - m.ypos_y)
    10                                           
    11         2       688821 344410.5     10.7      filtered = m[m.dist <= distance**2]
    12                                           
    13         2         2715   1357.5      0.0      filtered = filtered[['frame_idx','id_x', 'id_y']]
    14         2            2      1.0      0.0      return filtered

Total time: 35.5469 s
File: <ipython-input-89-d3df72123a10>
Function: get_close_bees_kd at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def get_close_bees_kd(df, distance):
     2                                           
     3         2         1335    667.5      0.0      df_close = DataFrame()
     4                                           
     5         2          768    384.0      0.0      gr = df.groupby('frame_idx')
     6                                           
     7      2050       761253    371.3      2.1      for i, group in gr:
     8      2048      2113139   1031.8      5.9          xy_coordinates = group[['xpos', 'ypos']].values
     9      2048       702913    343.2      2.0          tree = spatial.KDTree(xy_coordinates, leafsize=20)
    10      2048     23770087  11606.5     66.9          result = tree.query_pairs(distance)
    11      2048      3031548   1480.2      8.5          l = [[i,group['id'].iat[a], group['id'].iat[b]] for a,b in result]
    12      2048      5165871   2522.4     14.5          df_close = df_close.append(DataFrame(l, columns=['frame_idx', 'id_x', 'id_y']))

In [ ]:
#side0 =  77.5146484375
#side1 = 56.1884765625